### Working with Procedures dataset

## 1. Importing modules

In [ ]:
import numpy as np
import pandas as pd

#### Allowing all rows and columns to be displayed

In [ ]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

## 2. Importing and evaluating data

### Note that data was loaded in R using package from parseRPDR   
https://cran.r-project.org/web/packages/parseRPDR/vignettes/Using_parseRPDR.html   
##### Importing data from .txt files
##### Looking at df attributes (# of rows/observations, # of variables/columns)

In [ ]:
labs_first = pd.read_csv(r'First/lab.csv')

In [ ]:
labs_second = pd.read_csv(r'Second/lab.csv')
labs_third = pd.read_csv(r'Third/lab.csv')
labs_fourth = pd.read_csv(r'Fourth/lab.csv')

In [ ]:
print('Lab result:\n',labs_first['lab_group'].value_counts())

In [ ]:
print(len(labs_first))
print(len(labs_second))
print(len(labs_third))
print(len(labs_fourth))

In [ ]:
labs_merged = labs_first.append([labs_second, labs_third, labs_fourth])

In [ ]:
labs_merged = pd.read_csv(r'labs_merged.csv')

##### Column names, non-missing values, and data type (dtype)

In [ ]:
print('Lab group:\n',labs_merged['lab_group'].value_counts(dropna=False))

lab_loinc, lab_testID, lab_descript are contained in lab_group   
lab_result_abn, lab_spec, lab_accession is not relevant

In [ ]:
print('Lab group:\n',labs_merged['lab_spec'].value_counts(dropna=False))

lab_result_txt needs to be cleaned

In [ ]:
print('Lab result:\n',labs_merged['lab_result_txt'].value_counts(dropna=False).loc[lambda x : x>4])

In [ ]:
labs_merged = labs_merged.iloc[:, [0, 3, 4, 8, 9, 10, 11, 12]]

lab_result has characters and lab_result_txt needs to be cleaned

Ashwin prepared code to clean these fields, uploaded to slack lab_processing

First applying to lab_result

In [ ]:
labs_merged['lab_result'] = labs_merged['lab_result'].astype(str)

In [ ]:
labs_merged['PSA'] = ""

In [ ]:
import re
def psa_find(row):
    row = row.replace(",", "")
    if "Free PSA Ratio is only calculated" in row or "Not applicable" in row or "Test not performed" in row or "Unable to calculate" in row or "RATIO IS NOT CALCULATED" in row or "Results do not belong" in row or "not calculated" in row or "not reported" in row or "should be" in row or "CLINICAL USEFULNESS IS NOT ESTABLISHED" in row:
        return ""
    res = re.findall(r'PSA\s*\(Total\):\s*\d*\.?\d+', row, flags=re.IGNORECASE)
    if not res:
        res = re.findall(r'PSA TOTAL\s*\d*\.?\d+', row, flags=re.IGNORECASE)
        if not res:
            res = re.findall(r'\d*\.?\d+\s*(?:ng\/ml|mcg\/l|mcg\/ml)',row, flags=re.IGNORECASE)
            if not res:
                res = re.findall(r'PSA\s*\d*\.?\d+', row)
                if not res:
                    res = re.findall(r'PSA\s*=\s*\d*\.?\d+', row)
                    if not res:
                        res = re.findall(r'(?:ng\/ml|mcg\/l|mcg\/ml)\s*=?:?\s*\d*\.?\d+', row, flags=re.IGNORECASE)
                        if not res:
                            res = re.findall(r'^>?\d*\.?\d+', row)
                            if not res:
                                if ";" in row:
                                    res = re.findall(r'^\d*\.?\d+', row.split(";")[0])
                                if not res:
                                    return ""
    return re.findall(r'\d*\.?\d+', res[0])[0]

In [ ]:
def undetect(row):
    row = row.replace(",", "")
    res = re.findall(r'^<', row)
    if not res:
        res = re.findall(r'Less Than \d*\.?\d+', row, flags=re.IGNORECASE)
    if res:
        return True
    return False

In [ ]:
def greater(row):
    row = row.replace(",", "")
    res = re.findall(r'^PSA:?\s*(?:>|GREATER THAN)\s*\d*\.?\d+', row, flags=re.IGNORECASE)
    res2 = re.findall(r'^>\s*\d*\.?\d+', row, flags=re.IGNORECASE)
    if res or res2:
        return True
    return False

In [ ]:
def percent(row):
    row = row.replace(",", "")
    res = re.findall(r'PSA\s*\d*\.?\d+\s*%', row, flags=re.IGNORECASE)
    if not res:
        res = re.findall(r'^\d*\.?\d+\s*%', row)
        if not res:
            res = re.findall(r'% Free PSA:\s*\d*\.?\d+', row)
            if not res:
                res = re.findall(r'PSA FREE\s*\d*\.?\d+', row, flags=re.IGNORECASE)
                if not res:
                    return ""
    return re.findall(r'\d*\.?\d+', res[0])[0]

In [ ]:
def percent_from_lab_group(row):
    return "Free" in row['lab_group']

In [ ]:
labs_merged['PSA'] = labs_merged.apply (lambda row: psa_find(row['lab_result']), axis=1)

In [ ]:
labs_merged['undetect'] = labs_merged.apply (lambda row: undetect(row['lab_result']), axis=1)
labs_merged.loc[labs_merged.undetect == True, 'PSA'] = 0.009

In [ ]:
labs_merged['greater'] = labs_merged.apply (lambda row: greater(row['lab_result']), axis=1)

In [ ]:
labs_merged['Percent Free'] = labs_merged.apply(lambda row: percent_from_lab_group(row), axis=1)
for index, row in labs_merged.iterrows():
    res = percent(row['lab_result'])
    if res:
        labs_merged.loc[index, 'PSA'] = res
        labs_merged.loc[index, 'Percent Free'] = True

Now applying to lab_result_txt

In [ ]:
labs_merged['lab_result_txt'] = labs_merged['lab_result_txt'].astype(str)

In [ ]:
labs_merged['PSA'] = labs_merged.apply (lambda row: psa_find(row['lab_result_txt']) if row['PSA'] == "" else row["PSA"], axis=1)

In [ ]:
labs_merged['undetect'] = labs_merged.apply (lambda row: undetect(row['lab_result_txt']) if row['undetect'] == False else row["undetect"], axis=1)
labs_merged.loc[labs_merged.undetect == True, 'PSA'] = 0.009

In [ ]:
labs_merged['greater'] = labs_merged.apply (lambda row: greater(row['lab_result_txt']) if row['greater'] == False else row["greater"], axis=1)

In [ ]:
for index, row in labs_merged.iterrows():
    res = percent(row['lab_result_txt'])
    if res:
        labs_merged.loc[index, 'PSA'] = res
        labs_merged.loc[index, 'Percent Free'] = True
labs_merged["Total PSA"] = ~labs_merged['Percent Free']

In [ ]:
labs_merged.rename({'ID_MERGE': 'EMPI'}, axis=1, inplace=True)

In [ ]:
labs_cl = labs_merged[["EMPI", "time_lab_result", "lab_group", "PSA", "undetect", "greater", "Percent Free", "Total PSA", 'lab_result', 'lab_result_txt']]

In [ ]:
labs_cl.to_csv("labs_cl.csv")

In [ ]:
labs_cl = labs_merged.iloc[:, [2, 3, 4, 8, 9, 10, 11, 12]]

In [ ]:
labs_cl.to_csv(r'labs_cl.csv', index=False)

In [ ]:
labs_merged['datetime'] = pd.to_datetime(labs_merged['time_lab_result'], infer_datetime_format=True)

In [ ]:
labs_merged['latestDate'] = labs_merged.groupby('EMPI').datetime.transform('max')

In [ ]:
latest = labs_merged.copy()

In [ ]:
latest = latest.drop_duplicates(subset=['EMPI'])

In [ ]:
latest = latest[["EMPI", "latestDate"]]

In [ ]:
latest.to_csv("lab_latest_date.csv")

## Review next dataset